##The Set-Up

In [1]:
#Installs and Imports all correct Libraries and Packages

# --- Install dependencies (LangChain version) ---
!pip install -q "requests>=2.32.5,<3.0.0" langchain langchain-community sentence-transformers faiss-cpu transformers pypdf docx2txt gradio
    #This "requests" thing was built by AI because I was getting errors
# --- Imports ---
import os, glob
import numpy as np

from sentence_transformers import SentenceTransformer
import faiss
from transformers import pipeline

# LangChain document loaders
from langchain_community.document_loaders import TextLoader, PyPDFLoader, Docx2txtLoader
import gradio as gr



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 79.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.5/329.5 kB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 72.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


RAG:

In [2]:
#Uploads documents for RAG

DOCS_DIR = "BA_docs"   # folder where you put your PDFs

raw_docs = []

print(f" Loading PDFs from: {DOCS_DIR}\n")

pdf_paths = glob.glob(os.path.join(DOCS_DIR, "*.pdf"))

print("Found PDF files:")
for p in pdf_paths:
    print(" -", p)

print("\n--- Loading pages ---")
for path in pdf_paths:
    print(f"Loading: {path}")
    docs = PyPDFLoader(path).load()
    print(f" → Loaded {len(docs)} pages")
    raw_docs.extend(docs)

print("\n=====================================")
print("FINAL: Total pages loaded:", len(raw_docs))
print("=====================================")


 Loading PDFs from: BA_docs

Found PDF files:
 - BA_docs/BUSNUS.pdf
 - BA_docs/BUSNAHandbook.pdf
 - BA_docs/BRD Template.pdf
 - BA_docs/Stakeholders.pdf
 - BA_docs/FuncNonFunc.pdf

--- Loading pages ---
Loading: BA_docs/BUSNUS.pdf
 → Loaded 2 pages
Loading: BA_docs/BUSNAHandbook.pdf
 → Loaded 433 pages
Loading: BA_docs/BRD Template.pdf
 → Loaded 2 pages
Loading: BA_docs/Stakeholders.pdf
 → Loaded 20 pages
Loading: BA_docs/FuncNonFunc.pdf
 → Loaded 7 pages

FINAL: Total pages loaded: 464


In [3]:
!$ pip install -q langchain_text_splitters
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,       # larger chunks for long BA materials
    chunk_overlap=200,     # small overlap for context continuity
    length_function=len,
)

chunked_docs = text_splitter.split_documents(raw_docs)

print(f"Original pages loaded: {len(raw_docs)}")
print(f"Total chunks created: {len(chunked_docs)}")

# Preview the first chunk
chunked_docs[0]

/bin/bash: line 1: $: command not found
Original pages loaded: 464
Total chunks created: 859


Document(metadata={'producer': 'Microsoft® PowerPoint® 2010', 'creator': 'Microsoft® PowerPoint® 2010', 'creationdate': '2015-10-05T20:40:28-05:00', 'title': 'PowerPoint Presentation', 'author': 'Duarte;Inc. 2014', 'moddate': '2015-10-05T20:40:28-05:00', 'source': 'BA_docs/BUSNUS.pdf', 'total_pages': 2, 'page': 0, 'page_label': '1'}, page_content='\u200bUser Story Narrative \n\u200bAs a <Consumer of the value> \n\u200bI want to <Functionality (What)> \nso that <Value/benefit (Why)> \n \n\u200bTips \n• Write stories from the end user \nperspective (user can be a system) \n• Understand what the user wants \nand why (not how) \n• Don’t assume just one kind of user \nof the system \n• Don’t confuse user stories with \ntasks.  Stories describe what the \nuser wants and why.  Tasks are how \nwe will achieve the goal/benefit of \nthe story. \n \n\u200bAcceptance Criteria \n\u200bScenario 1: <summary of test> \n\u200bGiven <precondition or assumption> \n\u200bAnd <additional precondition> \n\u

In [4]:
#Builds FAISS and Loads Embedding Models
!pip install --upgrade langchain langchain-community

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = FAISS.from_documents(chunked_docs, embedding_model)

print("Number of embedded chunks in vectorstore:", len(vectorstore.docstore._dict))

/tmp/ipython-input-1427712966.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Number of embedded chunks in vectorstore: 859


In [5]:
SAVE_DIR = "ba_faiss_store"
vectorstore.save_local(SAVE_DIR)

print("Vectorstore saved to:", SAVE_DIR)

Vectorstore saved to: ba_faiss_store


##Pretrained Model:

In [6]:
#Reload FAISS vectorstore
!pip install --upgrade langchain langchain-community
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

SAVE_DIR = "ba_faiss_store"

vectorstore = FAISS.load_local(
    SAVE_DIR,
    embedding_model,
    allow_dangerous_deserialization=True
)

print("Vectorstore loaded successfully.")

Vectorstore loaded successfully.


In [7]:
query = "WHat is a functional requirement"

results = vectorstore.similarity_search_with_score(query, k=3)
results

[(Document(id='3cd595f1-5e6c-4a91-af4b-84b40e29aabf', metadata={'producer': 'PyPDF', 'creator': 'PyPDF', 'creationdate': '2009-09-06T05:01:34+02:00', 'author': 'Howard Podeswa', 'keywords': 'ISBN-13:    9781598635652', 'moddate': '2009-09-06T05:02:08+02:00', 'subject': 'Course Technology PTR', 'title': "The Business Analyst's Handbook", 'source': 'BA_docs/BUSNAHandbook.pdf', 'total_pages': 433, 'page': 345, 'page_label': '346'}, page_content='Glossary of BA Terms 325\nTerm What They Say\n(Standard or Guideline)\nWhat It Means to the BA\nFunctional requirement\nGap Analysis\nGate Review\n“Functional requirements define the\nsoftware functionality the developers\nmust build into the product to\nenable users to accomplish their\ntasks.”\n108\n“Typically, requirements are broadly\ncategorized as functional or non-\nfunctional. Functional requirements\ndescribe capabilities the system will\nbe able to perform in terms of\nbehaviors or operations—a specific\nsystem action or response.\nFunct

In [8]:
#Alot of this cell used AI to help generate after lang chain update
from transformers import pipeline
from langchain_community.llms import HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# --- LLM pipeline ---
gen = pipeline(
    "text2text-generation",
    model="google/flan-t5-large", #Switched to T5 large because base model could process as well or as fast
    max_length=80,
    truncation=True,
)
llm = HuggingFacePipeline(pipeline=gen)

# --- Retriever from your existing vectorstore ---
retriever = vectorstore.as_retriever(search_kwargs={"k": 6})

# Format docs into a single context string
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# --- Prompt template that actually uses {context} and {question} ---
template = """

Context:
{context}

Question:
{question}

Answer the question in your own words. Follow these rules:
- Use at most 3 bullet points.
- Each bullet is one short, simple sentence.
- Do NOT copy sentences or long phrases from the context.
- Do NOT mention the words "context", "document", or page numbers.

Answer:
"""

prompt = PromptTemplate.from_template(template)

def filter_to_functional(docs):
    new_docs = []
    for d in docs:
        text = d.page_content.lower()
        if "functional requirement" in text:
            new_docs.append(d)
    return new_docs if new_docs else docs  # fall back if empty


# --- LCEL chain ---
qa_chain = (
    {"context": retriever | filter_to_functional | format_docs,
     "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


def compress_to_short_bullets(text: str, max_bullets: int = 2):
    # Split into rough sentences
    parts = text.replace("\n", " ").split(".")
    sentences = [p.strip() for p in parts if p.strip()]

    # 1) Prefer sentences about functional requirements only
    functional_sentences = [
        s for s in sentences
        if "functional" in s.lower() and "non-functional" not in s.lower()
    ]

    # 2) If we found some, use those; otherwise fall back to all sentences
    if functional_sentences:
        sentences = functional_sentences

    # 3) Take the first few sentences only
    sentences = sentences[:max_bullets]

    # 4) Turn into short bullets
    bullets = [f"- {s}" for s in sentences]

    return "\n".join(bullets)


    # Take the first few sentences only
    sentences = sentences[:max_bullets]

    # Turn into short bullets
    bullets = [f"- {s}" for s in sentences]

    # Join back
    return "\n".join(bullets)


def ba_answer(question: str) -> str:
    long_answer = qa_chain.invoke(question)
    short_answer = compress_to_short_bullets(long_answer, max_bullets=2)
    return short_answer


print(ba_answer("What are functional requirements?"))
print(ba_answer("What is a user story?"))
print(ba_answer("What is the purpose of a BRD?"))

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
/tmp/ipython-input-1720427249.py:14: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=gen)


- Functional requirement Gap Analysis Gate Review “Functional requirements define the software functionality the developers must build into the product to enable users to accomplish their tasks
- Functional requirements describe capabilities the system will be able to perform in terms of behaviors or operations—a specific system action or response
- Write stories from the end user perspective (user can be a system) • Understand what the user wants and why (not how) • Don’t confuse user stories with tasks
- Stories describe what the user wants and why
- Business Requirements Document is the consolidated docu- mentation of all of the requirements (capabilities that a solution must provide or conditions that it must meet) that stem from the business
- Ideally, the BRD is an electronic assembly of smaller documents, each housed separately in a repository


In [13]:
#API call from hugging face

import requests
def finance_api(question):
    url = "https://ayush2917-finance-news-api.hf.space/chat"
    r = requests.post(url, json={"message": question})
    return r.json()["response"]


['**Summary**', 'Dublin, Dec', '01, 2025 (GLOBE NEWSWIRE) -- The "Precision Fermentation Market - Global Industry Size, Share, Trends, Opportunity, and Forecast, 2020-2030F" report has been added to ResearchAndMarkets.…...', '', '**Investment Recommendations for Indian Investors**', '1. Reliance Industries offers growth in diverse sectors. [Read more: http://economictimes.com/markets/stocks/news/reliance-industries-q3-results-2025/]', '2. TCS provides stable returns in IT sector. [Read more: http://economictimes.com/markets/stocks/news/tcs-q3-results-2025/]', '3. HDFC Bank ensures consistent performance for investors. [Read more: http://moneycontrol.com/news/hdfc-bank-q3-results-2025/]', '4. Infosys grows with IT outsourcing trends. [Read more: http://livemint.com/news/infosys-q3-results-2025/]', '5. Adani Ports benefits from infrastructure growth. [Read more: http://livemint.com/news/adani-ports-q3-results-2025/]']


##Gradio GUI
(Much of this was AI generated)

In [14]:
def process_query(user_query: str):
    # Call the ba_answer function
    ba_response = ba_answer(user_query)

    # Call the finance_api function
    finance_response_list = finance_api(user_query)

    # Format the finance_response (list of strings) into a single string
    finance_response_formatted = "\n".join(finance_response_list)

    return ba_response, finance_response_formatted

# Test the function (optional, but good for verification)
# query_test = "What are functional requirements and current stock trends for IT Companies?"
# ba_out, finance_out = process_query(query_test)
# print("\n--- BA Output ---")
# print(ba_out)
# print("\n--- Finance Output ---")
# print(finance_out)

In [ ]:
iface = gr.Interface(
    fn=process_query,
    inputs=gr.Textbox(lines=2, placeholder="Enter your query here..."),
    outputs=[
        gr.Textbox(label="Business Assistant Response"),
        gr.Textbox(label="Finance API Response")
    ],
    title="Business/Finance News AI Assistant",
    description="Ask a question to get insights from both Business Assistant and Finance API."
)

iface.launch(debug=True, share=False)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.
* To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>